Required packages: vfb-connect and python-catmaid (pymaid & navis)

In [1]:
!pip install vfb-connect --upgrade
!pip install python-catmaid --upgrade

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
  Using cached decorator-4.4.2-py2.py3-none-any.whl (9.2 kB)
  Attempting uninstall: decorator
    Found existing installation: decorator 5.0.6
    Not uninstalling decorator at /shared-libs/python3.7/py-core/lib/python3.7/site-packages, outside environment /root/venv
    Can't uninstall 'decorator'. No files were found to uninstall.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


### A note on using these notebooks
This is designed as an interactive tutorial. Feel free to add code cells below each example to try out variations of your own.

### How to find neurons across datasets

VirtualFlyBrain integrates images and connectomics profiles of neurons from many sources.  It classifies and records their properties using a standard, queryable classification ([The Drosophila Anatomy Ontology](https://www.ebi.ac.uk/ols/ontologies/fbbt)). This standardises the names of neuron types across sources, so you don't need to worry about differences in nomenclature uses and supports queries for neurons by their classification. 

In [2]:
# Import libs and initialise API objects
from vfb_connect.cross_server_tools import VfbConnect
import pandas as pd
vc = VfbConnect()

import pymaid
import navis

navis.set_pbars(jupyter=False)
pymaid.set_pbars(jupyter=False)

# Connect to the VFB CATMAID server hosting the FAFB data
rm = pymaid.connect_catmaid(server="https://fafb.catmaid.virtualflybrain.org/", api_token=None, max_threads=10)

# Test call to see if connection works 
print(f'Server is running CATMAID version {rm.catmaid_version}')


INFO  : Global CATMAID instance set. Caching is ON. (pymaid)
Server is running CATMAID version 2020.02.15-990-gb5aac7433


### Finds neurons by type (classification) across datasets

We can use the `vc.get_instances` method in combination with theNname or Symbol of a neuron type on VFB to find individual neurons from multiple sources.

Use the search tool on [VFB](http://virtualflybrain.org) to find neuron types by name or synonym:

<img src="https://user-images.githubusercontent.com/112839/109564687-535c9380-7ad9-11eb-80e9-5a5bc21cd915.png" width=30% height=30%>
<img src="https://user-images.githubusercontent.com/112839/109565128-eac1e680-7ad9-11eb-9649-1ec55a298a1b.png" width=30% height=30%>

Use either the full name or the Symbol to query for neurons:

In [3]:
DA3adPN = vc.get_instances("DA3_adPN", summary=True)
pd.DataFrame.from_records(DA3adPN)

,label,symbol,id,tags,parents_label,parents_id,data_source,accession,templates,dataset,license
0,Uniglomerular mALT DA3 adPN#L1 (FAFB:2449792),,VFB_0010123h,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult antennal lobe projection neuron DA3 adPN,FBbt_00100384,catmaid_fafb,2449792,JRC2018Unisex,None,
1,Uniglomerular mALT DA3 adPN#R2 (FAFB:65465),,VFB_00101240,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult antennal lobe projection neuron DA3 adPN,FBbt_00100384,catmaid_fafb,65465,adult brain template JFRC2|JRC2018Unisex,None,
2,Uniglomerular mALT DA3 adPN#R1 (FAFB:57349),,VFB_00101239,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult antennal lobe projection neuron DA3 adPN,FBbt_00100384,catmaid_fafb,57349,JRC2018Unisex|adult brain template JFRC2,None,
3,DA3_adPN_R (FlyEM-HB:1703683361),,VFB_jrchjtdn,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult antennal lobe projection neuron DA3 adPN,FBbt_00100384,neuprint_JRC_Hemibrain_1point1,1703683361,JRC2018Unisex|JRC_FlyEM_Hemibrain,None,
4,Uniglomerular mALT DA3 adPN#L2 (FAFB:2350853),,VFB_0010122u,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult antennal lobe projection neuron DA3 adPN,FBbt_00100384,catmaid_fafb,2350853,JRC2018Unisex,None,
5,DA3_adPN_R (FlyEM-HB:666135689),,VFB_jrchjtdo,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult antennal lobe projection neuron DA3 adPN,FBbt_00100384,neuprint_JRC_Hemibrain_1point1,666135689,JRC_FlyEM_Hemibrain|JRC2018Unisex,None,


### Find neurons by location

We can use the same method to search for neurons by location, using simple queries.  See [Semantic query guide](https://vfb-connect.readthedocs.io/en/latest/Semantic_queries.html) for more info.

In [4]:
# Find neurons by location. The following query works across multiple data sources and both sides of the brain.  
# Results may be incomplete & may include minor overlap inferred from low synapse counts

neurons_in_DA3 = vc.get_instances("'neuron' that 'overlaps' some 'antennal lobe glomerulus DA3'", summary=True)
neurons_in_DA3_tab = pd.DataFrame.from_records(neurons_in_DA3)
neurons_in_DA3_tab[0:5]


Running query: FBbt:00005106 that RO:0002131 some FBbt:00003934
Query URL: http://owl.virtualflybrain.org/kbs/vfb/instances?object=FBbt%3A00005106+that+RO%3A0002131+some+FBbt%3A00003934&prefixes=%7B%22FBbt%22%3A+%22http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FFBbt_%22%2C+%22RO%22%3A+%22http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FRO_%22%2C+%22BFO%22%3A+%22http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FBFO_%22%7D&direct=False
Query results: 158


,label,symbol,id,tags,parents_label,parents_id,data_source,accession,templates,dataset,license
0,ORN_D (FlyEM-HB:1639368236),,VFB_jrchk121,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult olfactory receptor neuron Or69a,FBbt_00067034,neuprint_JRC_Hemibrain_1point1,1639368236,JRC2018Unisex|JRC_FlyEM_Hemibrain,None,
1,lLN1_b_R (FlyEM-HB:5813078440),,VFB_jrchk8a7,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult antennal lobe local neuron type 1 lLN,FBbt_20003740,neuprint_JRC_Hemibrain_1point1,5813078440,JRC2018Unisex|JRC_FlyEM_Hemibrain,None,
2,ORN_DA1_L (FlyEM-HB:1672311578),,VFB_jrchk13h,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult olfactory receptor neuron Or67d,FBbt_00067031,neuprint_JRC_Hemibrain_1point1,1672311578,JRC2018Unisex|JRC_FlyEM_Hemibrain,None,
3,M_vPNml53_R (FlyEM-HB:5813008771),,VFB_jrchk0y3,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult multiglomerular antennal lobe projection...,FBbt_20003830,neuprint_JRC_Hemibrain_1point1,5813008771,JRC_FlyEM_Hemibrain|JRC2018Unisex,None,
4,ORN_DA3 (FlyEM-HB:1734718131),,VFB_jrchk18m,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult olfactory receptor neuron Or23a,FBbt_00067033,neuprint_JRC_Hemibrain_1point1,1734718131,JRC_FlyEM_Hemibrain|JRC2018Unisex,None,


In [5]:
# Find local interneurons (intrinsic neurons) of the AL, overlapping DA3:

local_in_DA3 = vc.get_instances("'local interneuron of adult antennal lobe' that 'overlaps' some 'antennal lobe glomerulus DA3'",
                                summary=True)
local_in_DA3_tab = pd.DataFrame.from_records(local_in_DA3)
local_in_DA3_tab

Running query: FBbt:00007390 that RO:0002131 some FBbt:00003934
Query URL: http://owl.virtualflybrain.org/kbs/vfb/instances?object=FBbt%3A00007390+that+RO%3A0002131+some+FBbt%3A00003934&prefixes=%7B%22FBbt%22%3A+%22http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FFBbt_%22%2C+%22RO%22%3A+%22http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FRO_%22%2C+%22BFO%22%3A+%22http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FBFO_%22%7D&direct=False
Query results: 53


,label,symbol,id,tags,parents_label,parents_id,data_source,accession,templates,dataset,license
0,lLN1_b_R (FlyEM-HB:1610530558),,VFB_jrchk8ac,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult antennal lobe local neuron type 1 lLN,FBbt_20003740,neuprint_JRC_Hemibrain_1point1,1610530558,JRC2018Unisex|JRC_FlyEM_Hemibrain,None,
1,lLN2T_c(Tortuous)_R (FlyEM-HB:1671292719),,VFB_jrchk8bo,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult antennal lobe lateral local neuron 2T,FBbt_00049816,neuprint_JRC_Hemibrain_1point1,1671292719,JRC2018Unisex|JRC_FlyEM_Hemibrain,None,
2,lLN2S(Star)_R (FlyEM-HB:1732995501),,VFB_jrchk8ba,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult antennal lobe lateral local neuron 2S,FBbt_00049815,neuprint_JRC_Hemibrain_1point1,1732995501,JRC2018Unisex|JRC_FlyEM_Hemibrain,None,
3,lLN2T_e(Tortuous)_R (FlyEM-HB:1699974843),,VFB_jrchk8br,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult antennal lobe lateral local neuron 2T,FBbt_00049816,neuprint_JRC_Hemibrain_1point1,1699974843,JRC_FlyEM_Hemibrain|JRC2018Unisex,None,
4,lLN1_b_R (FlyEM-HB:5813054622),,VFB_jrchk8a8,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult antennal lobe local neuron type 1 lLN,FBbt_20003740,neuprint_JRC_Hemibrain_1point1,5813054622,JRC2018Unisex|JRC_FlyEM_Hemibrain,None,
5,v2LN30_R (FlyEM-HB:1671620613),,VFB_jrchk8e8,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult antennal lobe local neuron type 30 v2LN,FBbt_20003756,neuprint_JRC_Hemibrain_1point1,1671620613,JRC2018Unisex|JRC_FlyEM_Hemibrain,None,
6,lLN1_b_R (FlyEM-HB:1642623277),,VFB_jrchk8ad,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult antennal lobe local neuron type 1 lLN,FBbt_20003740,neuprint_JRC_Hemibrain_1point1,1642623277,JRC2018Unisex|JRC_FlyEM_Hemibrain,None,
7,lLN2T_b(Tortuous)_R (FlyEM-HB:1640572741),,VFB_jrchk8bl,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult antennal lobe lateral local neuron 2T,FBbt_00049816,neuprint_JRC_Hemibrain_1point1,1640572741,JRC2018Unisex|JRC_FlyEM_Hemibrain,None,
8,lLN1_c_R (FlyEM-HB:1824101645),,VFB_jrchk8ag,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult antennal lobe local neuron type 1 lLN,FBbt_20003740,neuprint_JRC_Hemibrain_1point1,1824101645,JRC_FlyEM_Hemibrain|JRC2018Unisex,None,
9,lLN2T_d(Tortuous)_R (FlyEM-HB:1667251683),,VFB_jrchk8bp,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult antennal lobe lateral local neuron 2T,FBbt_00049816,neuprint_JRC_Hemibrain_1point1,1667251683,JRC2018Unisex|JRC_FlyEM_Hemibrain,None,


In [6]:
# Find neurons by dataset/paper - on CATMAID

bates = pymaid.find_neurons(annotations='Paper: Bates and Schlegel et al 2020')
bates

INFO  : Found 583 neurons matching the search parameters (pymaid)


,type,name,skeleton_id,n_nodes,n_connectors,n_branches,n_leafs,cable_length,soma,units
0,CatmaidNeuron,Uniglomerular mALT DA1 lPN 57316 2863105 ML,2863104,NA,NA,NA,NA,NA,NA,1 nanometer
1,CatmaidNeuron,Uniglomerular mALT DA3 adPN 57350 HG,57349,NA,NA,NA,NA,NA,NA,1 nanometer
...,...,...,...,...,...,...,...,...,...,...
581,CatmaidNeuron,Multiglomerular mlALT vPN VM7d+VM5d+DC4+6 LTS ...,4624378,NA,NA,NA,NA,NA,NA,1 nanometer
582,CatmaidNeuron,Uniglomerular mALT DL2d adPN 57342 ML,57341,NA,NA,NA,NA,NA,NA,1 nanometer


In [7]:
# Inspect what datasets are available on VFB

ds = vc.neo_query_wrapper.get_datasets(summary=True)
ds_tab = pd.DataFrame.from_records(ds)
ds_tab.sort_values(by=['id'])

,label,symbol,id,tags,description,miniref,FlyBase,PMID,DOI
36,EM L1 Andrade et al. 2019,,Andrade2019,Entity|Individual|DataSet,[L1 EM reconstructed neurons from Andrade et a...,"Andrade et al., 2019, Curr. Biol. 29(3): 412--...",FBrf0241389,30661802,10.1016/j.cub.2018.12.012
55,MBONs and split-GAL4 lines that target them (A...,,Aso2014,Entity|Individual|DataSet,[],"Aso et al., 2014, eLife 3: e04577",FBrf0227179,25535793,10.7554/eLife.04577
73,split-GAL4 lines for dopaminergic neurons (Aso...,,AsoRubin2016,Entity|Individual|DataSet,[For comparison of the properties of memories ...,"Aso and Rubin, 2016, eLife 5: e16135",FBrf0233230,27441388,10.7554/eLife.16135
59,EM FAFB Bates and Schlegel et al 2020,,BatesSchlegel2020,Entity|Individual|DataSet,[FAFB EM reconstructed neurons from Bates and ...,"Bates and Schlegel et al., 2020",,,10.1016/j.cub.2020.06.042
12,Larval olfactory system neurons - EM (Berk2016),,Berck2016,Entity|Individual|DataSet,"[Berck, Khandelwal et al. 2016]","Berck et al., 2016, eLife 5: e14859",FBrf0232785,27177418,10.7554/eLife.14859
...,...,...,...,...,...,...,...,...,...
14,Lee lab adult brain lineage clone image set,,Yu2013,Entity|Individual|DataSet,[An exhaustive set of lineage clones covering ...,"Yu et al., 2013, Curr. Biol. 23(8): 633--643",FBrf0221412,23541733,10.1016/j.cub.2013.02.057
64,"EM L1 Zarin, Mark et al. 2019",,Zarin2019,Entity|Individual|DataSet,"[L1 EM reconstructed neurons from Zarin, Mark ...","Zarin, Mark et al., 2019",,,10.1101/617977
6,EM FAFB Zheng et al 2018,,Zheng2018,Entity|Individual|DataSet,[FAFB EM reconstructed neurons from Zheng et a...,"Zheng et al., 2018, Cell 174(3): 730--743.e22",FBrf0239557,30033368,10.1016/j.cell.2018.06.019
79,EM FAFB Zheng et al 2020,,Zheng2020,Entity|Individual|DataSet,[FAFB EM reconstructed neurons from Zheng et a...,Zheng et al. 2020,,,10.1101/2020.04.17.047167


You can also find linked datasets directly on 

In [8]:
sayin_tab = pd.DataFrame.from_records(vc.get_instances_by_dataset('Sayin2019', summary=True))
sayin_tab

,label,symbol,id,tags,parents_label,parents_id,data_source,accession,templates,dataset,license
0,OA-VPM4 (FAFB:1191261),,VFB_001001dq,Entity|Octopaminergic|Adult|Anatomy|has_image|...,octopaminergic VPM4 neuron,FBbt_00110152,catmaid_fafb,1191261,JRC2018Unisex,None,
1,OA-VPM3 (FAFB:1329078),,VFB_001001dr,Entity|Octopaminergic|Adult|Anatomy|has_image|...,octopaminergic VPM3 neuron,FBbt_00110151,catmaid_fafb,1329078,JRC2018Unisex,None,


In [9]:
# Find connections between neurons by type across datasets.
# Note: results are limited by how detailed annotation currently is in the DB.  
    
vc.get_connected_neurons_by_type(upstream_type='GABAergic neuron', downstream_type='adult descending neuron', weight=10).sort_values(['weight'],ascending=False)

,upstream_neuron_id,upstream_neuron_name,weight,downstream_neuron_id,downstream_neuron_name,upstream_class,downstream_class,up_data_source,up_accession,down_source,down_accession
4,VFB_001011rg,Uniglomerular mlALT VP4 vPN bilateral#R1 (FAFB...,156,VFB_001011rk,DNp44 (FAFB:3094186),adult antennal lobe projection neuron VP4+ vPN,adult lateral horn PV9a1 neuron|descending neu...,None,None,None,None
21,VFB_00101152,Uniglomerular t5-mlALT VP4 vPN#R1 (FAFB:1149173),142,VFB_001011rk,DNp44 (FAFB:3094186),adult antennal lobe projection neuron VP4 vPN,adult lateral horn PV9a1 neuron|descending neu...,None,None,None,None
17,VFB_jrchk7ji,VP4_vPN(mlALT)_R (FlyEM-HB:634759240),65,VFB_jrchjthu,DNp44_R (FlyEM-HB:542751938),adult antennal lobe projection neuron VP4 vPN,descending neuron of the posterior brain DNp44,None,None,None,None
7,VFB_jrchk7jh,VP4+_vPN(mlALT)_R (FlyEM-HB:543010474),50,VFB_jrchjthu,DNp44_R (FlyEM-HB:542751938),adult antennal lobe projection neuron VP4+ vPN,descending neuron of the posterior brain DNp44,None,None,None,None
9,VFB_jrchk7jh,VP4+_vPN(mlALT)_R (FlyEM-HB:543010474),47,VFB_jrchjthj,DNp25_R (FlyEM-HB:451689001),adult antennal lobe projection neuron VP4+ vPN,descending neuron of the posterior brain DNp25,None,None,None,None
12,VFB_jrchk8dp,oviIN_R (FlyEM-HB:423101189),43,VFB_jrchk8dm,oviDNa_R (FlyEM-HB:550655668),oviposition inhibitory neuron,oviposition descending neuron a,None,None,None,None
3,VFB_jrchk8dq,oviIN_L (FlyEM-HB:485934965),40,VFB_jrchk8do,oviDNb_R (FlyEM-HB:519949044),oviposition inhibitory neuron,oviposition descending neuron b,None,None,None,None
6,VFB_jrchk8dp,oviIN_R (FlyEM-HB:423101189),36,VFB_jrchk8do,oviDNb_R (FlyEM-HB:519949044),oviposition inhibitory neuron,oviposition descending neuron b,None,None,None,None
22,VFB_001013vv,oviIN (FAFB:13325634),35,VFB_001013vm,oviDNa#2 (FAFB:1875104),oviposition inhibitory neuron,oviposition descending neuron a,None,None,None,None
8,VFB_jrchk8dp,oviIN_R (FlyEM-HB:423101189),35,VFB_jrchk8dn,oviDNb(PDM15)_L (FlyEM-HB:642763374),oviposition inhibitory neuron,oviposition descending neuron b,None,None,None,None
